In [ ]:
# main reference
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

import random
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

raw = ["I feel hungry.	나는 배가 고프다.",
       "Pytorch is very easy.	파이토치는 매우 쉽다.",
       "Pytorch is a framework for deep learning.	파이토치는 딥러닝을 위한 프레임워크이다.",
       "Pytorch is very clear to use.	파이토치는 사용하기 매우 직관적이다."]

SOS_token = 0
EOS_token = 1


class Vocab:
    def __init__(self):
        self.vocab2index = {"<SOS>": SOS_token, "<EOS>": EOS_token}
        self.index2vocab = {SOS_token: "<SOS>", EOS_token: "<EOS>"}
        self.vocab_count = {}
        self.n_vocab = len(self.vocab2index)

    def add_vocab(self, sentence):
        for word in sentence.split(" "):
            if word not in self.vocab2index:
                self.vocab2index[word] = self.n_vocab
                self.vocab_count[word] = 1
                self.index2vocab[self.n_vocab] = word
                self.n_vocab += 1
            else:
                self.vocab_count[word] += 1                         #word가 vocab_count에 있으면 value를 +1


def filter_pair(pair, source_max_length, target_max_length):        # pair는 문장.
    return len(pair[0].split(" ")) < source_max_length and len(pair[1].split(" ")) < target_max_length  #source_max_length보다 작은지 and target_max_length보다 작은지 확인


def preprocess(corpus, source_max_length, target_max_length):
    print("reading corpus...")
    pairs = []
    for line in corpus:
        pairs.append([s for s in line.strip().lower().split("\t")])                         #strip()은 양면의 공백제거. append하는 대상은 리스트다. 아래의 예시를 봐라.
    print("Read {} sentence pairs".format(len(pairs)))

    pairs = [pair for pair in pairs if filter_pair(pair, source_max_length, target_max_length)]     # if True면ㅇㅇ.
    print("Trimmed to {} sentence pairs".format(len(pairs)))

    source_vocab = Vocab()
    target_vocab = Vocab()

    print("Counting words...")
    for pair in pairs:
        source_vocab.add_vocab(pair[0])
        target_vocab.add_vocab(pair[1])
    print("source vocab size =", source_vocab.n_vocab)
    print("target vocab size =", target_vocab.n_vocab)

    return pairs, source_vocab, target_vocab


class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)  # input_size는 source text를 이루는 corpus에서 사용되는 단어의 개수
                                                                # hidden_size만큼 줄어들은 vector로 표현하는 matrix를 embedding이라 한다.  
        self.gru = nn.GRU(hidden_size, hidden_size)             # 배치사이즈와 시퀀스는 자동으로 인식한다.

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)                    #embedding의 결과의 shape을 바꾸는 거임. (1, 1, hidden_size)
        x, hidden = self.gru(x, hidden)
        return x, hidden


class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)                             
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):#아래의 tensorize에서 들어가는 x의 형태 ["I", "feel", "hungry", "<EOS>"] → [1, 2, 3, 4] 인덱스 형태가 됨. 그러고는 view로 (4,1)이 됨.
        x = self.embedding(x).view(1, 1, -1)           #1,1,hidden_size
        x, hidden = self.gru(x, hidden)                 #gru의 출력은 1,1,hidden_size
        x = self.softmax(self.out(x[0]))                #x[0]은 1,hiddensize를 넣는거임.
        return x, hidden


def tensorize(vocab, sentence):
    indexes = [vocab.vocab2index[word] for word in sentence.split(" ")]     #vocab.vocab2index라는 dictionary의 [word]에 해당하는 인덱스 추출.
    indexes.append(vocab.vocab2index["<EOS>"])
    return torch.Tensor(indexes).long().to(device).view(-1, 1)
#tensorize에서 들어가는 x의 형태 ["I", "feel", "hungry", "<EOS>"] → [1, 2, 3, 4] 인덱스 형태가 됨. 그러고는 view로 (4,1)이 됨


def train(pairs, source_vocab, target_vocab, encoder, decoder, n_iter, print_every=1000, learning_rate=0.01):
    loss_total = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    training_batch = [random.choice(pairs) for _ in range(n_iter)]          # n_iter만큼 문장을 넣음.
    training_source = [tensorize(source_vocab, pair[0]) for pair in training_batch]
    training_target = [tensorize(target_vocab, pair[1]) for pair in training_batch]

    criterion = nn.NLLLoss()

    for i in range(1, n_iter + 1):
        source_tensor = training_source[i - 1]                  # i=1부터이니.
        target_tensor = training_target[i - 1]

        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        source_length = source_tensor.size(0)
        target_length = target_tensor.size(0)

        loss = 0

        for enc_input in range(source_length):
            _, encoder_hidden = encoder(source_tensor[enc_input], encoder_hidden)

        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        decoder_hidden = encoder_hidden

        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # teacher forcing

        loss.backward()

        encoder_optimizer.step()        #순서는 딱히 상관없음.
        decoder_optimizer.step()

        loss_iter = loss.item() / target_length                 #문장마다 단어수가 다르기 때문에에
        loss_total += loss_iter

        if i % print_every == 0:
            loss_avg = loss_total / print_every
            loss_total = 0
            print("[{} - {}%] loss = {:05.4f}".format(i, i / n_iter * 100, loss_avg))


def evaluate(pairs, source_vocab, target_vocab, encoder, decoder, target_max_length):
    for pair in pairs:
        print(">", pair[0])
        print("=", pair[1])
        source_tensor = tensorize(source_vocab, pair[0])
        source_length = source_tensor.size()[0]
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)            #

        for ei in range(source_length):
            _, encoder_hidden = encoder(source_tensor[ei], encoder_hidden)

        decoder_input = torch.tensor([[SOS_token]], device=device).long()               #이미 존재하는 데이터를 tesor로 바꾸고 싶을 때 torch.tenseor()를 사용
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(target_max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            _, top_index = decoder_output.data.topk(1) #topk(k) 함수는 주어진 텐서에서 상위 k개의 값과 그에 대응하는 인덱스를 반환합니다. 
                                                       #여기서 k=1이므로 가장 큰 값 하나만을 반환
                                                       #topk(1)의 결과로 반환되는 값은 두 가지입니다:
                                                       #1. 가장 큰 값들 (topk()에서 얻은 값들)
                                                       #2. 그 값들의 인덱스 (가장 큰 값들이 어디에 위치하는지)
                                                       #참고로  top_index는 2D 텐서 (1,1)형태의 ㅇㅇ. ([[]])
            if top_index.item() == EOS_token:
                decoded_words.append("<EOS>")
                break
            else:
                decoded_words.append(target_vocab.index2vocab[top_index.item()])

            decoder_input = top_index.squeeze().detach()    #decoder_input을 top_index로 바꿈.
                                                            
                                                            #top_index는 (1,1)형태인데 squeeze() 함수는 텐서에서 차원이 1인 축을 제거. (1,)의 1D 텐서로 변경됨.
                                                            #detach() method는 gradient의 전파를 멈추는 역할을 함.
                                                            #PyTorch에서는 기본적으로 연산을 할 때마다 계산 그래프가 자동으로 추적됩니다. 
                                                            #그런데, 평가 단계에서는 기울기를 계산할 필요가 없기 때문에, 계산 그래프가 계속 추적되지 않도록 끊어
                                                            #메모리 사용을 줄인다.

                                                            # detach는 주어진 Tensor 객체에 대한 연산의 결과로 생성된 새로운 Tensor 객체가 있을 때, 
                                                            # 이 새로운 Tensor 객체를 사용하여 추가적인 계산을 수행하고자 할 때, 
                                                            # 기존 Tensor 객체의 연산 그래프와의 의존성을 제거하여 메모리 사용량을 줄이고 계산 속도를 향상시키는 데 유용

                                                            

        predict_words = decoded_words
        predict_sentence = " ".join(predict_words)
        print("<", predict_sentence)
        print("")


SOURCE_MAX_LENGTH = 10
TARGET_MAX_LENGTH = 12
load_pairs, load_source_vocab, load_target_vocab = preprocess(raw, SOURCE_MAX_LENGTH, TARGET_MAX_LENGTH)
print(random.choice(load_pairs))

enc_hidden_size = 16
dec_hidden_size = enc_hidden_size
enc = Encoder(load_source_vocab.n_vocab, enc_hidden_size).to(device)
dec = Decoder(dec_hidden_size, load_target_vocab.n_vocab).to(device)

train(load_pairs, load_source_vocab, load_target_vocab, enc, dec, 5000, print_every=1000)
evaluate(load_pairs, load_source_vocab, load_target_vocab, enc, dec, TARGET_MAX_LENGTH)

reading corpus...
Read 4 sentence pairs
Trimmed to 4 sentence pairs
Counting words...
source vocab size = 17
target vocab size = 13
['pytorch is very clear to use.', '파이토치는 사용하기 매우 직관적이다.']
[1000 - 20.0%] loss = 0.7416
[2000 - 40.0%] loss = 0.1081
[3000 - 60.0%] loss = 0.0338
[4000 - 80.0%] loss = 0.0181
[5000 - 100.0%] loss = 0.0122
> i feel hungry.
= 나는 배가 고프다.
< 나는 배가 고프다. <EOS>

> pytorch is very easy.
= 파이토치는 매우 쉽다.
< 파이토치는 매우 쉽다. <EOS>

> pytorch is a framework for deep learning.
= 파이토치는 딥러닝을 위한 프레임워크이다.
< 파이토치는 딥러닝을 위한 프레임워크이다. <EOS>

> pytorch is very clear to use.
= 파이토치는 사용하기 매우 직관적이다.
< 파이토치는 사용하기 매우 직관적이다. <EOS>



참고) torch.Tensor와 torch.tensor의 차이

https://velog.io/@minchoul2/torch.Tensor%EC%99%80-torch.tensor%EC%9D%98-%EC%B0%A8%EC%9D%B4

torch.tensor

int 입력시 int 그대로 입력
입력받은 데이터를 새로운 메모리 공간에 복사해 Tensor 객체 생성 (call by value)
torch.Tensor

int 입력시 float으로 변환
데이터 입력 시(Tensor 객체로) 입력 받은 메모리 공간을 그대로 사용 (call by reference)
데이터 입력 시(list나 numpy 로) 입력 받은 값을 복사하여 Tensor 객체 생성(call by value)

In [4]:
a="word in sentence"
a.split(" ")

['word', 'in', 'sentence']

In [5]:
"I feel hungry.	나는 배가 고프다.".strip().lower().split('\t')

['i feel hungry.', '나는 배가 고프다.']

In [1]:
a=[1,2,3,4]
a.1

SyntaxError: invalid syntax (2420263525.py, line 2)

In [ ]:
exit()

: 